# Description and docs

- https://colab.research.google.com/drive/1_1kxEiuh_Zt-WdxUUA-TfIWkDT9zLOeT?usp=sharing#scrollTo=TYfiaegE6JVo
- https://www.youtube.com/watch?v=8mpycgjlQ5I
- https://kernc.github.io/backtesting.py/doc/backtesting/#gsc.tab=0

In [2]:
#Importamos las librerias
from backtesting import Backtest, Strategy
from backtesting.lib import crossover,plot_heatmaps


import numpy
import talib
import pandas as pd

In [3]:
# get data in satoshi
df = pd.read_csv('data/BTCUSDT.csv', parse_dates=True, index_col=0)
data = (df / 1e6).assign(Volume=df.Volume * 1e6)  # μBTC OHLC prices

# Estrategias

In [58]:
class RSIcross(Strategy):
  rsi_period = 17 
  slPercent = 0.99
 
  def init(self):
    self.rsi = self.I(talib.RSI,self.data.Close,self.rsi_period)

  def next(self):
    price = self.data.Close[-1]

    if self.rsi[-1] < 30:
      self.position.close()

    if self.position.is_long:
      return
    
    
    self.buy(sl = price * self.slPercent)




In [45]:
#  Medias moviles https://www.youtube.com/watch?v=8mpycgjlQ5I
class SMAcross(Strategy):
    # Define the two MA lags as *class variables*
    # for later optimization
    n1 = 10
    n2 = 20
    
    def init(self):
        # Precompute the two moving averages
        self.sma1 = self.I(SMA, self.data.Close, self.n1)
        self.sma2 = self.I(SMA, self.data.Close, self.n2)
    
    def next(self):
        # If sma1 crosses above sma2, close any existing
        # short trades, and buy the asset
        if crossover(self.sma1, self.sma2):
            self.position.close()
            self.buy()

        # Else, if sma1 crosses below sma2, close any existing
        # long trades, and sell the asset
        elif crossover(self.sma2, self.sma1):
            self.position.close()
            self.sell()



# Run BackTest

In [14]:
#Ejecutamos el backtest

# bt = Backtest(data,SMAcross,cash = 100000,exclusive_orders=True)
# bt = Backtest(data,RSIcross,cash = 100000)


In [60]:
# 1 mes a 15m

bt = Backtest(data,RSIcross,cash = 100)
# stats = bt.run()

stats = bt.optimize(
    rsi_period = range(10,30,1),
    slPercent = [0.9,0.91,0.92,0.93,0.94,0.95,0.96,0.97,0.98,0.99],
    maximize = "Return [%]",
)

stats

Start                     2023-07-21 00:00:00
End                       2024-06-16 00:30:00
Duration                    331 days 00:30:00
Exposure Time [%]                   99.949701
Equity Final [$]                   250.032482
Equity Peak [$]                    272.939187
Return [%]                         150.032482
Buy & Hold Return [%]              122.313499
Return (Ann.) [%]                  173.836201
Volatility (Ann.) [%]              134.114782
Sharpe Ratio                         1.296175
Sortino Ratio                        6.388164
Calmar Ratio                         8.026159
Max. Drawdown [%]                  -21.658703
Avg. Drawdown [%]                   -1.803025
Max. Drawdown Duration       68 days 12:30:00
Avg. Drawdown Duration        2 days 00:31:00
# Trades                                  629
Win Rate [%]                        33.068362
Best Trade [%]                       25.55245
Worst Trade [%]                     -5.147146
Avg. Trade [%]                    

In [62]:
stats["_strategy"]

<Strategy RSIcross(rsi_period=14,slPercent=0.99)>

In [ ]:


bt = Backtest(data[1:3000],RSIcross ,cash = 100)
stats = bt.run()


In [8]:
bt = Backtest(data,RSIcross,cash = 100)
stats = bt.run()

stats = bt.optimize(
    rsi_period = range(10,30,1),
    slPercent = [0.9,0.91,0.92,0.93,0.94,0.95,0.96,0.97,0.98,0.99],
    maximize = "Return [%]",
)

stats

KeyboardInterrupt: 

In [24]:
#Grafico con los resultados
bt.plot()

/root/trading/.venv/lib/python3.10/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/root/trading/.venv/lib/python3.10/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/root/trading/.venv/lib/python3.10/site-packages/backtesting/_plotting.py:455: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  df2 = (df.assign(_width=1).set_index('datetime')
/root/trading/.venv/lib/python3.10/site-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
/root/trading/.venv/lib/python3.10/site-packages/backtesting/_plotting.py:659: UserW

GridPlot(id='p1729', ...)